<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>ChatGPT and Friends</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI
import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

Compiler    : Clang 15.0.0 (clang-1500.1.0.2.5)
OS          : Darwin
Release     : 23.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: 1f87e80538ad172ebadf16b8ffe7f1e01f363ed6

watermark : 2.4.3
json      : 2.0.9
tqdm      : 4.66.1
pandas    : 2.2.0
termcolor : 2.4.0
matplotlib: 3.8.2
numpy     : 1.26.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

We start by getting a list of supported models.

In [5]:
model_list = client.models.list().data

In total we have 25 models

In [6]:
len(model_list)

27

Along with some information about each model...

In [7]:
model_list[:3]

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system')]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo-preview
gpt-4-vision-preview
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 7.08 ms, sys: 1.95 ms, total: 9.03 ms
Wall time: 1.28 s


Which produces a response object

In [10]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a simple object

In [11]:
pprint(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is kryptonite, a mineral from his home planet Krypton. Exposure to kryptonite weakens him and can even kill him.", role='assistant', function_call=None, tool_calls=None))


The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response.choices[0].message.content

"Superman's weakness is kryptonite, a mineral from his home planet Krypton. Exposure to kryptonite weakens him and can even kill him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [13]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 7.6 ms, sys: 1.62 ms, total: 9.22 ms
Wall time: 6.76 s


And we can access each of the answers individually int he choices list

In [14]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
In the fictional world of Superman, there are several different types of Kryptonite, each with different effects on the superhero. Here are some of the most well-known types:

1. Green Kryptonite: This is the most common and widely known form of Kryptonite. It is radioactive remnants of Superman's home planet, Krypton. When Superman is exposed to Green Kryptonite, it weakens him, drains his powers, and even causes intense pain. Prolonged exposure to it can be fatal.

2. Red Kryptonite: Red Kryptonite has unpredictable effects on Superman. Every time he comes into contact with it, it causes temporary changes to his abilities, personality, or behavior. These changes are often erratic and can last from hours to days.

3. Blue Kryptonite: Blue Kryptonite is specifically harmful to Bizarro, a twisted clone of Superman. It weakens Bizarro's powers and can cause him pain and discomfort. It has no effect on the real Superman.

4. Gold Kryptonite: Gold Kryptonite permanently removes S

In [15]:
response.usage

CompletionUsage(completion_tokens=1118, prompt_tokens=17, total_tokens=1135)

# Temperature

In [16]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 7.01 ms, sys: 1.85 ms, total: 8.85 ms
Wall time: 16.3 s


In [17]:
print(response.choices[0].message.content)

Once upon a time in a small village, there was a hardworking boy named Jack. He lived with his elderly mother in a small, cozy cottage.

Jack was known for his kindness and willingness to help others. Every day, he would go to the nearby forest and chop wood to earn a livelihood for his mother and himself. One afternoon, while Jack was working, he became intrigued by a faint melody that seemed to be coming from deep within the forest.

Guided by curiosity, Jack followed the mesmerizing sound and after what seemed like a never-ending journey, discovered an enchanted lake, shimmering under the setting sun. Floating on the water was a beautiful but saddened swan.

As soon as their eyes met, the swan realized that Jack's open-mindedness and pure heart weren't common qualities found in most people. With a gentle voice, the swan told Jack of his unfortunate circumstances. Legend foretold that under the light of a golden moonrise, a magical herb would grow at the bottom of the lake. Consuming

In [18]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 6.2 ms, sys: 1.58 ms, total: 7.78 ms
Wall time: 7.29 s


In [19]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the heart of a lush forest, there lived a young girl named Lily. She was known for her kind heart and adventurous spirit. One sunny morning, as she was exploring the woods, she stumbled upon a wounded bird with a broken wing.

Lily carefully picked up the bird and cradled it in her hands. She knew she had to help it, so she decided to take it home. With great care, she fashioned a small nest for the bird and gently placed it inside. Lily named the bird Ruby, after its vibrant red feathers.

Days turned into weeks, and Lily tirelessly tended to Ruby's needs. She fed her, kept her warm, and even sang lullabies to comfort her. Slowly but surely, Ruby's wing began to heal. Lily's dedication and love had worked wonders.

One day, as Lily was preparing to release Ruby back into the wild, a sudden storm swept through the village. The wind howled, and rain poured down in torrents. Lily worried about Ruby's safety, knowing she couldn't fly yet.

W

## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [20]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This sudden change in direction means we don't have enough time to complete the entire project for the client.


# Formatted output

In [21]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 8.06 ms, sys: 1.59 ms, total: 9.64 ms
Wall time: 3.85 s


In [22]:
json_output = response.choices[0].message.content

In [23]:
output = json.loads(json_output)

In [24]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tablespoons sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/4 teaspoon salt',
 '3/4 cup buttermilk',
 '1/4 cup milk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh blueberries',
 'Maple syrup, for serving']

In [25]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.',
 'In a separate bowl, whisk together the buttermilk, milk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. Do not overmix; the batter should be lumpy.',
 'Gently fold in the blueberries.',
 'Heat a non-stick griddle or skillet over medium heat. Lightly grease with butter or cooking spray.',
 'Scoop 1/4 cup of batter onto the griddle for each pancake. Cook until bubbles form on the surface, then flip and cook for another 1-2 minutes, until golden brown.',
 'Remove pancakes from the griddle and repeat with the remaining batter.',
 'Serve the blueberry pancakes warm with maple syrup.']

# Translation

In [26]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [27]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [28]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [29]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [30]:
print(response.choices[0].message.content)

Fruit,Color,Taste
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour and bitter


In [31]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [32]:
print(response.choices[0].message.content)

Pounits are bright green fruits found on the recently discovered planet Goocrux. They have a more savory taste rather than being sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>